In [1]:
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from pprint import pprint
import json
import requests
import numpy as np
import pandas as pd

from src.params import *
from src.nws import NWSClient
from src.nws import fahrenheit_to_celsius, celsius_to_fahrenheit, load_clis


In [2]:
nws = NWSClient()

# Initial Questions

In the long term, I'd like to build a model that can accurately predict the probability of the high temp being in a certain 2 degree range given the available information. Using the link from @BigJ, I now have 1-minute observations on the 4 thermometers MIA, NYC, MDW, and AUS. The first questions:
1. @BigJ claims that temps in CLIs are exactly the max of these one-minute readings. Verify this.
2. Given (1) is true, what is the variation between the linear interpolation of the 5 minute readings and the 1 minute readings? I'd like to see this both as a timeseries and as a histogram. Same for the precise 1 hour readings. Does it follow a particular distribution?

In [4]:
# Verify that CLI maxes are same as max of 1-minute observations
# Load all the CLIS
station = StationID.MIA

clis = load_clis(station)

pprint([cli.without_raw_text() for cli in clis])

[CLI(issuance_time=datetime.datetime(2024, 10, 25, 4, 25, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
     issuing_office='MFL',
     summary_date=datetime.date(2024, 10, 24),
     raw_text='...',
     max_temp=84,
     max_temp_time=datetime.datetime(2024, 10, 24, 14, 22, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
     min_temp=73,
     min_temp_time=datetime.datetime(2024, 10, 24, 5, 31, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
     avg_temp=79,
     valid_time=None),
 CLI(issuance_time=datetime.datetime(2024, 11, 11, 16, 22, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
     issuing_office='MFL',
     summary_date=datetime.date(2024, 11, 11),
     raw_text='...',
     max_temp=87,
     max_temp_time=datetime.datetime(2024, 11, 11, 11, 17, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
     min_temp=74,
     min_temp_time=datetime.datetime(2024, 11, 11, 3, 50, tzinfo=<DstTzInfo 'America

In [5]:
# Now load all the 1-minute observations within the same time range
start_date = min(cli.summary_date for cli in clis)
end_date = max(cli.summary_date for cli in clis)


